# Multi-Angle QAOA Example

This notebook demonstrates the multi-angle QAOA functionality, which allows problems, mixers, and initial states to use multiple parameters per layer.

## Overview

Standard QAOA uses one parameter per layer:
- One γ (gamma) for the problem Hamiltonian
- One β (beta) for the mixer

Multi-angle QAOA generalises this:
- **Multi-angle mixer**: Each qubit gets its own β parameter
- **Parameterized initial state**: The initial state has its own optimizable parameters

The flat angle array format used throughout is:
```
[init_0, ..., init_{n-1},  # initial state params (0 for standard)
 gamma_0, beta_0_0, beta_0_1, ...,  # layer 0
 gamma_1, beta_1_0, beta_1_1, ...,  # layer 1
 ...]
```

For single-parameter components, this reduces to the familiar `[gamma_0, beta_0, gamma_1, beta_1, ...]` format.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from qiskit_aer import Aer

from qaoa import QAOA
from qaoa.problems import MaxKCutBinaryPowerOfTwo
from qaoa.mixers import X, XMultiAngle
from qaoa.initialstates import Plus, PlusParameterized

backend = Aer.get_backend('qasm_simulator')

# Create a small MaxCut problem: triangle graph
V = np.arange(0, 3, 1)
E = [(0, 1, 1.0), (1, 2, 1.0), (0, 2, 1.0)]
G = nx.Graph()
G.add_nodes_from(V)
G.add_weighted_edges_from(E)

nx.draw(G, with_labels=True)
plt.title('MaxCut: Triangle Graph')
plt.show()

problem = MaxKCutBinaryPowerOfTwo(G, 2)
print(f'Number of qubits: {problem.N_qubits}')

## 1. Standard QAOA (Backward Compatibility)

The standard X mixer has `get_num_parameters() = 1` (one β per layer),
and the Plus initial state has `get_num_parameters() = 0` (no parameters).

In [ ]:
# Standard QAOA - identical to existing usage
mixer_standard = X()
init_standard = Plus()

print(f'X mixer parameters per layer: {mixer_standard.get_num_parameters()}')

qaoa_standard = QAOA(problem, mixer_standard, init_standard, backend=backend, shots=1024)

# Build the parameterized circuit at depth 1
qaoa_standard.createParameterizedCircuit(depth=1)
print(f'n_gamma={qaoa_standard.n_gamma}, n_beta={qaoa_standard.n_beta}, n_init={qaoa_standard.n_init}')

# Flat angle array: [gamma_0, beta_0]
angles_d1 = np.array([0.5, 0.3])
hist = qaoa_standard.hist(angles_d1, shots=1024)
print(f'Sample histogram: {dict(list(sorted(hist.items(), key=lambda x: -x[1]))[:5])}')

# Interpolation from depth 1 to depth 2
angles_d2 = qaoa_standard.interp(angles_d1)
print(f'Interpolated angles (depth 1 -> 2): {angles_d2}')

## 2. Multi-Angle Mixer (XMultiAngle)

The `XMultiAngle` mixer assigns one independent β parameter per qubit.
For a problem with N qubits, there are N β parameters per layer.

Flat angle array format for depth 1:
`[gamma_0, beta_0_qubit0, beta_0_qubit1, ..., beta_0_qubitN]`

In [ ]:
mixer_multi = XMultiAngle()
init_standard = Plus()

qaoa_multi = QAOA(problem, mixer_multi, init_standard, backend=backend, shots=1024)

qaoa_multi.createParameterizedCircuit(depth=1)
print(f'n_gamma={qaoa_multi.n_gamma}, n_beta={qaoa_multi.n_beta} (one per qubit), n_init={qaoa_multi.n_init}')
print(f'Total parameters at depth 1: {qaoa_multi.n_init + 1*(qaoa_multi.n_gamma + qaoa_multi.n_beta)}')

# Flat angle array: [gamma_0, beta_0_0, beta_0_1, ..., beta_0_{N-1}]
n_params = qaoa_multi.n_init + 1 * (qaoa_multi.n_gamma + qaoa_multi.n_beta)
angles_multi = np.random.uniform(0, np.pi, size=n_params)
print(f'Random angles: {angles_multi}')

hist_multi = qaoa_multi.hist(angles_multi, shots=1024)
print(f'Sample histogram: {dict(list(sorted(hist_multi.items(), key=lambda x: -x[1]))[:5])}')

# Interpolation
angles_d2_multi = qaoa_multi.interp(angles_multi)
print(f'Interpolated angles (depth 1 -> 2): {angles_d2_multi}')
print(f'Length: {len(angles_d2_multi)} (should be 2 * (n_gamma + n_beta) = {2*(qaoa_multi.n_gamma+qaoa_multi.n_beta)})')

## 3. Parameterized Initial State (PlusParameterized)

The `PlusParameterized` initial state applies Hadamard gates followed by parameterized RZ rotations.
Each qubit gets its own phase parameter that is optimized jointly with the QAOA parameters.

Flat angle array format for depth 1:
`[init_phase_0, ..., init_phase_{N-1}, gamma_0, beta_0]`

Note: Initial state parameters appear first in the flat array.

In [ ]:
mixer_standard = X()
init_param = PlusParameterized()  # one phase per qubit by default

qaoa_param_init = QAOA(problem, mixer_standard, init_param, backend=backend, shots=1024)

qaoa_param_init.createParameterizedCircuit(depth=1)
print(f'n_gamma={qaoa_param_init.n_gamma}, n_beta={qaoa_param_init.n_beta}, n_init={qaoa_param_init.n_init} (one per qubit)')
print(f'Total parameters at depth 1: {qaoa_param_init.n_init + 1*(qaoa_param_init.n_gamma + qaoa_param_init.n_beta)}')

# Flat angle array: [init_0, ..., init_{N-1}, gamma_0, beta_0]
n_params = qaoa_param_init.n_init + 1 * (qaoa_param_init.n_gamma + qaoa_param_init.n_beta)
angles_init = np.zeros(n_params)
print(f'Initial angles (zeros): {angles_init}')

hist_init = qaoa_param_init.hist(angles_init, shots=1024)
print(f'Sample histogram: {dict(list(sorted(hist_init.items(), key=lambda x: -x[1]))[:5])}')

# Interpolation: init params stay unchanged
angles_d2_init = qaoa_param_init.interp(angles_init)
print(f'Interpolated angles (depth 1 -> 2): {angles_d2_init}')
print(f'Init params unchanged: {angles_d2_init[:qaoa_param_init.n_init]}')

## 4. Combined Multi-Angle Usage

Combining `XMultiAngle` mixer with `PlusParameterized` initial state.

In [ ]:
mixer_multi = XMultiAngle()
init_param = PlusParameterized()

qaoa_full_multi = QAOA(problem, mixer_multi, init_param, backend=backend, shots=1024)

depth = 2
qaoa_full_multi.createParameterizedCircuit(depth=depth)
n_gamma = qaoa_full_multi.n_gamma
n_beta = qaoa_full_multi.n_beta
n_init = qaoa_full_multi.n_init
n_per_layer = n_gamma + n_beta
n_total = n_init + depth * n_per_layer

print(f'n_gamma={n_gamma}, n_beta={n_beta}, n_init={n_init}')
print(f'Parameters per layer: {n_per_layer}')
print(f'Total parameters at depth {depth}: {n_total}')
print(f'  - Initial state: {n_init}')
print(f'  - Per layer: {n_per_layer} x {depth} layers = {depth*n_per_layer}')

# Random angles
angles_full = np.random.uniform(0, np.pi, size=n_total)
hist_full = qaoa_full_multi.hist(angles_full, shots=1024)
print(f'\nSample histogram: {dict(list(sorted(hist_full.items(), key=lambda x: -x[1]))[:5])}')

## 5. Parameter Format Summary

The flat angle array format is:
```
[init_0, ..., init_{n_init-1},           # Initial state params (n_init values)
 gamma_{0,0}, ..., gamma_{0,n_gamma-1},   # Layer 0 problem params
 beta_{0,0}, ..., beta_{0,n_beta-1},      # Layer 0 mixer params
 gamma_{1,0}, ..., gamma_{1,n_gamma-1},   # Layer 1 problem params
 beta_{1,0}, ..., beta_{1,n_beta-1},      # Layer 1 mixer params
 ...
]
```

**Backward compatibility**: For the default case (standard X mixer + Plus initial state):
- `n_init = 0`, `n_gamma = 1`, `n_beta = 1`
- This gives the classic `[gamma_0, beta_0, gamma_1, beta_1, ...]` format

In [ ]:
# Demonstrate backward compatibility
qaoa_standard2 = QAOA(
    MaxKCutBinaryPowerOfTwo(G, 2), X(), Plus(),
    backend=backend, shots=1024
)
qaoa_standard2.createParameterizedCircuit(depth=2)

# Classic format still works
classic_angles = np.array([0.5, 0.3, 0.4, 0.2])  # [g0, b0, g1, b1]
hist_classic = qaoa_standard2.hist(classic_angles, shots=512)
print(f'Standard QAOA depth=2 histogram: {dict(list(sorted(hist_classic.items(), key=lambda x: -x[1]))[:5])}')
print('Backward compatibility confirmed!')